In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
from strategy_v2.Strategy import *
from strategy_v2.Strategy.MVO import *
from strategy_v2.Strategy.MVO.AlphaModel import *
from strategy_v2.Strategy.MVO.RiskModel import *
from strategy_v2.TradingSubSystem import *
from strategy_v2.Portfolio import *
from strategy_v2.TransactionCost import *
from strategy_v2.Executor import *
from utils.data_helper import *
from utils.data import *
from utils.performance import *
from utils.ta import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
instruments = get_djia_tickers()

end_date = get_today()
start_date = pd.to_datetime(datetime(2024, 1, 3))
start_date = pd.to_datetime(datetime(2023, 2, 7))
max_leverage = 1
#vol_target = 0.25
vol_target = None

In [ ]:
portfolio = PortfolioStandard(
    capital=187224,
    name='MVOPortfolio',        
    rebalance_iter=RebalancerIter('0 0 * * Fri', 1),        
    tc_model=TransactionCostFutu(), 
    systems_style=SystemStyle.VERTICAL,
    systems=[        
        # Long terms signals => 60days return / diversified        
        TradingSubSystemSingle(vol_target=vol_target, instruments=instruments, strategy=[MeanVarianceOpt(alpha_model=RollingMean(60), risk_model=RollingMeanCovNeg(60), gamma=10, hhi=0.2, confidence=1, leverage=1)], max_leverage=max_leverage, offset=60),

        # Mid terms signals => 30days return / diversified        
        # TradingSubSystemSingle(vol_target=vol_target, instruments=insturments, strategy=[MeanVarianceOpt(alpha_model=RollingMean(30), risk_model=RollingMeanCovNeg(30), gamma=10, hhi=0.01, confidence=2)], max_leverage=max_leverage, offset=60),

        # Short terms signals => 5 and 10days returns / less diversified
        TradingSubSystemSingle(vol_target=vol_target, instruments=instruments, strategy=[MeanVarianceOpt(alpha_model=RollingMean(10), risk_model=RollingMeanCovNeg(10), gamma=30, hhi=0.4, confidence=0.5, leverage=1)], max_leverage=max_leverage, offset=60),         
        TradingSubSystemSingle(vol_target=vol_target, instruments=instruments, strategy=[MeanVarianceOpt(alpha_model=RollingMean(5), risk_model=RollingMeanCovNeg(5), gamma=30, hhi=0.4, confidence=0.5, leverage=1)], max_leverage=max_leverage, offset=60),         

        # Mean Revert signals => 2days RSI / concentrated
        # TradingSubSystemSingle(vol_target=vol_target, instruments=insturments, strategy=[MeanVarianceOpt(alpha_model=RSI(2,10), risk_model=ZeroCov(), gamma=0, hhi=0, confidence=0.5)], max_leverage=max_leverage, offset=200),
        # TradingSubSystemSingle(vol_target=vol_target, instruments=insturments, strategy=[MeanVarianceOpt(alpha_model=RSI(2,95), risk_model=ZeroCov(), gamma=0, hhi=0, confidence=-0.5)], max_leverage=max_leverage, offset=200),

        # Mean Revert signals => Double7 / concentrated
        TradingSubSystemSingle(vol_target=vol_target, instruments=instruments, strategy=[MeanVarianceOpt(alpha_model=Double7(5), risk_model=ZeroCov(), gamma=0, hhi=0, confidence=0.5)], max_leverage=max_leverage, offset=200),                        
    ]
)

portfolio.set_start_date(start_date)
portfolio.set_end_date(end_date)
portfolio.backtest_subsystems()
portfolio.optimize()
portfolio.backtest()
portfolio.rebalance()
portfolio.performance(show_all_rets=True)

[*********************100%***********************]  30 of 30 completed
2025-02-07 02:47:17,242 - TradingSubSystemSingle [MVO - RollingMean(60)|1] - INFO - Generating position for strategy MVO - RollingMean(60)|1 between 2023-02-06 and 2025-02-06......
 66%|██████▋   | 348/524 [03:49<05:57,  2.03s/it]

In [4]:
portfolio.portfolio_breakdown()
portfolio.instrument_breakdown()

In [5]:
portfolio.get_position_for_trade().tail(20)

2024-11-23 02:21:55,086 - Standard Portfolio ({self.name}) - CRITICAL - Portfolio target capital is not specified, use initial backtest capital of $187,224
2024-11-23 02:21:55,086 - Standard Portfolio ({self.name}) - INFO - Generate trade position based on target capital of $187,224


,AAPL,BRK-B,DXJ,EWY,META,NANC,NVDA,QQQ,SPY,TSLA
Date,,,,,,,,,,
2024-10-28,110.0,151.0,28.0,0.0,19.0,167.0,176.0,7.0,8.0,153.0
2024-10-29,110.0,151.0,28.0,0.0,19.0,167.0,176.0,7.0,8.0,153.0
2024-10-30,110.0,151.0,28.0,0.0,19.0,167.0,176.0,7.0,8.0,153.0
2024-10-31,110.0,151.0,28.0,0.0,19.0,167.0,176.0,7.0,8.0,153.0
2024-11-01,77.0,16.0,214.0,0.0,19.0,500.0,297.0,37.0,31.0,139.0
2024-11-04,77.0,16.0,214.0,0.0,19.0,500.0,297.0,37.0,31.0,139.0
2024-11-05,77.0,16.0,214.0,0.0,19.0,500.0,297.0,37.0,31.0,139.0
2024-11-06,77.0,16.0,214.0,0.0,19.0,500.0,297.0,37.0,31.0,139.0
2024-11-07,77.0,16.0,214.0,0.0,19.0,500.0,297.0,37.0,31.0,139.0


# Execute the Portfolio

In [41]:
executor = ExecutorFutu(is_test=False)
executor.set_portfolio(portfolio)
orders = executor.execute(px_interval='5m')
orders

2024-11-16 02:01:37,788 - ExecutorFutu - INFO - market: US
2024-11-16 02:01:37,791 - ExecutorFutu - INFO - Cancel all orders first before executing.....


2024-11-16 02:01:37,810 | 38190 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=1, host=127.0.0.1, port=11111, user_id=18214795
2024-11-16 02:01:37,993 | 38190 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=1


2024-11-16 02:01:40,999 - Standard Portfolio ({self.name}) - CRITICAL - Portfolio target capital is not specified, use initial backtest capital of $181,511
2024-11-16 02:01:41,012 - Standard Portfolio ({self.name}) - INFO - Generate trade position based on target capital of $181,511
2024-11-16 02:01:41,022 - ExecutorFutu - INFO - Execute Standard Portfolio ({self.name}) position based on 2024-11-15


2024-11-16 02:01:41,026 | 38190 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=2, host=127.0.0.1, port=11111, user_id=18214795


2024-11-16 02:01:41,075 - Futu - INFO - 10 Positions: US.TSLA, US.SPY, US.QQQ, US.NVDA, US.NANC, US.META, US.EWY, US.DXJ, US.BRK.B, US.AAPL


...

2024-11-16 02:02:08,532 - Futu - INFO - Placed Order: {'code': 'US.SPY', 'price': 585.39, 'qty': 21.0, 'trd_side': 'BUY', 'order_type': 'NORMAL', 'market': 'US', 'trd_env': 'REAL'}


2024-11-16 02:02:08,532 | 38190 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=11
2024-11-16 02:02:11,560 | 38190 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=12, host=127.0.0.1, port=11111, user_id=18214795


2024-11-16 02:02:11,797 - Futu - INFO - Placed Order: {'code': 'US.TSLA', 'price': 317.58, 'qty': 14.0, 'trd_side': 'BUY', 'order_type': 'NORMAL', 'market': 'US', 'trd_env': 'REAL'}


2024-11-16 02:02:11,798 | 38190 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=12


,code,stock_name,trd_side,order_type,order_status,order_id,qty,price,create_time,updated_time,...,remark,time_in_force,fill_outside_rth,aux_price,trail_type,trail_value,trail_spread,currency,portfolio,date
0,US.AAPL,苹果,BUY,NORMAL,SUBMITTING,8439034341417243995,52.0,224.69,2024-11-15 13:01:41,2024-11-15 13:01:41,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2024-11-15
0,US.BRK.B,伯克希尔-B,BUY,NORMAL,SUBMITTING,149688973569762307,35.0,470.56,2024-11-15 13:01:45,2024-11-15 13:01:45,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2024-11-15
0,US.DXJ,日本对冲股票ETF-WisdomTree,SELL,NORMAL,SUBMITTING,2136373786890907788,347.0,108.64,2024-11-15 13:01:48,2024-11-15 13:01:48,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2024-11-15
0,US.EWY,韩国ETF-iShares MSCI,SELL,NORMAL,SUBMITTING,7996314869145558076,33.0,56.16,2024-11-15 13:01:51,2024-11-15 13:01:51,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2024-11-15
0,US.META,Meta Platforms,SELL,NORMAL,SUBMITTING,9078952092628270356,16.0,555.68,2024-11-15 13:01:55,2024-11-15 13:01:55,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2024-11-15
0,US.NANC,UNUSUAL WHALES SUBVERSIVE DEMOCRATIC TRADING ETF,BUY,NORMAL,SUBMITTING,7245313963498147027,172.0,38.72,2024-11-15 13:01:58,2024-11-15 13:01:58,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2024-11-15
0,US.NVDA,英伟达,SELL,NORMAL,SUBMITTING,5352657674740523854,25.0,141.79,2024-11-15 13:02:01,2024-11-15 13:02:01,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2024-11-15
0,US.QQQ,纳指100ETF-Invesco QQQ Trust,BUY,NORMAL,SUBMITTING,8038741511280582075,2.0,496.29,2024-11-15 13:02:05,2024-11-15 13:02:05,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2024-11-15
0,US.SPY,SPDR 标普500指数ETF,BUY,NORMAL,SUBMITTING,6606620896327126665,21.0,585.39,2024-11-15 13:02:08,2024-11-15 13:02:08,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2024-11-15
0,US.TSLA,特斯拉,BUY,NORMAL,SUBMITTING,4739787441368654599,14.0,317.58,2024-11-15 13:02:11,2024-11-15 13:02:11,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2024-11-15


# Calibration Portfolios